In [1]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import pyro
import torch
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist
from pyro.infer.mcmc import NUTS
from pyro.infer.mcmc.api import MCMC
from pyro.optim import Adam

# torch.set_default_tensor_type(torch.cuda.FloatTensor)
tqdm.pandas()

In [2]:
torch.set_default_tensor_type(torch.FloatTensor)

In [ ]:
df = pd.read_csv('../data/games.csv', index_col=0)
df

In [ ]:
features = dict()

def parse_game(row):
    for color in ['White', 'Black']:
        player = row[color]
        
        if player in features:
            feats = features[player]
        else:
            feats = [0] * 7
            features[player] = feats
        
        feats[0] += 1
        feats[1] += row[color + 'Elo']
        feats[2] += abs(row['WhiteElo'] - row['BlackElo'])
        
        if row['Result'] == '1/2-1/2':
            feats[3] += 1
            if row['NMoves'] <= 30: 
                feats[4] += 1
                if color == 'White': feats[5] += 1
            feats[6] += row['NMoves']
        
_ = df.progress_apply(parse_game, axis=1)

In [ ]:
feature_df = pd.DataFrame.from_dict(features, orient='index', columns=['N', 'eloav', 'elodiff', 'd', 'd_short', 'd_short_W', 'd_length'])
feature_df['d_length'] = feature_df['d_length'] / feature_df['d']
for c in feature_df.columns[1:-1]:
    feature_df[c] = feature_df[c] / feature_df['N']

feature_df = feature_df[~(feature_df['N'] < 10)]
feature_df = feature_df[~(feature_df['d'] == 0)]
feature_df.to_csv('../data/features.csv')
feature_df

In [3]:
feature_df = pd.read_csv('../data/features.csv', index_col=0)
pca = PCA()
pca.fit(feature_df)
print(pca.explained_variance_ratio_)

[9.32012282e-01 5.91576033e-02 8.39702548e-03 4.32999356e-04
 7.50055667e-08 1.10704704e-08 3.36399836e-09]


In [ ]:
pyro.clear_param_store()

In [4]:
data = torch.from_numpy(feature_df.values)
n_factors = 3
# data = data.type(torch.float32).cuda()
pyro.enable_validation(True)
data.device

device(type='cpu')

In [ ]:

ss = 1000
# maybe change to torch.rand calls
def factor_model(data):
    W = pyro.param('W', torch.ones((data.shape[-1], n_factors)))
    mu = pyro.param('mu', torch.zeros(data.shape[-1]))
    theta = pyro.sample('theta', dist.HalfCauchy(torch.ones(data.shape[-1])).to_event(1))
    L_omega = pyro.sample('L_omega', dist.LKJCholesky(data.shape[-1], torch.ones(())))
    L_Omega = torch.mm(torch.diag(theta.sqrt()), L_omega)    
    i = 0
    with pyro.plate('data', len(data), subsample_size=ss) as ind:
        i += 1
        batch = data[ind]
        eps = pyro.sample(f'eps_{i}', dist.MultivariateNormal(torch.zeros(data.shape[-1]), scale_tril=L_Omega).expand([ss]))
        z = pyro.sample(f'lat_{i}', dist.MultivariateNormal(torch.zeros(n_factors), torch.eye(n_factors)).expand([ss]))
        pyro.sample(f'obs_{i}', dist.MultivariateNormal(
            torch.squeeze(torch.matmul(W, torch.unsqueeze(z, 2))) + mu + eps, torch.eye(data.shape[-1])).expand([ss]),
                    obs=batch)

In [ ]:
def factor_guide(data):
    W_ = pyro.param('W_', torch.ones((n_factors, data.shape[-1])))
    theta_ = pyro.sample('theta', dist.HalfCauchy(torch.ones(data.shape[-1])).to_event(1))
    L_omega_ = pyro.sample('L_omega', dist.LKJCholesky(data.shape[-1], torch.ones(())))
    L_Omega_ = torch.mm(torch.diag(theta_.sqrt()), L_omega_)
    i = 0 
    with pyro.plate('data', len(data), subsample_size=ss) as ind:
        i += 1
        batch = data[ind]
        eps = pyro.sample(f'eps_{i}', dist.MultivariateNormal(torch.zeros(data.shape[-1]), scale_tril=L_Omega_).expand([ss]))
        pyro.sample(f'lat_{i}', dist.MultivariateNormal(torch.squeeze(torch.matmul(W_, torch.unsqueeze(batch + eps, 2))), torch.eye(n_factors)).expand([ss]))


optimizer = Adam({"lr": 0.005, "betas": (0.9, 0.999)})
svi = SVI(factor_model, factor_guide, optimizer, loss=Trace_ELBO())

n_steps = 10000
losses = []
for i in tqdm(range(n_steps)):
    elbo = svi.step(data)
    losses.append(elbo)

In [5]:
# torch.multiprocessing.set_start_method('spawn')
# torch.multiprocessing.set_sharing_strategy("file_system")
data_small = data[:100]
def factor_model_mcmc(data):
    W = pyro.param('W', torch.ones((data.shape[-1], n_factors)))
    mu = pyro.param('mu', torch.zeros(data.shape[-1]))
    theta = pyro.sample('theta', dist.HalfCauchy(torch.ones(data.shape[-1])).to_event(1))
    L_omega = pyro.sample('L_omega', dist.LKJCholesky(data.shape[-1], torch.ones(())))
    L_Omega = torch.mm(torch.diag(theta.sqrt()), L_omega)    
    
    for i in pyro.plate('data', len(data)):
        eps = pyro.sample(f'eps_{i}', dist.MultivariateNormal(torch.zeros(data.shape[-1]), scale_tril=L_Omega))
        z = pyro.sample(f'lat_{i}', dist.MultivariateNormal(torch.zeros(n_factors), torch.eye(n_factors)))
        pyro.sample(f'obs_{i}', dist.MultivariateNormal(W @ z + mu + eps, torch.eye(data.shape[-1])), obs=data[i])


nuts_kernel = NUTS(factor_model_mcmc, jit_compile=False, step_size=1e-5)
MCMC(
    nuts_kernel,
    num_samples=200,
    warmup_steps=100,
    num_chains=1
).run(data_small)

Warmup:   0%|                                                                                                                          | 0/300 [00:00, ?it/s]

RuntimeError: Expected b and A to have the same dtype, but found b of type Double and A of type Float instead.

In [ ]:
plt.figure(figsize=(5, 2))
plt.plot(losses)
plt.xlabel("SVI step")
plt.ylabel("ELBO loss")

print(losses[-1], min(losses))

In [ ]:
list(pyro.get_param_store().items())

In [ ]:
names_to_ids = {n : i for i, n in enumerate(feature_df.index)}
names = names_to_ids.keys()
pruned_df = df[df['White'].isin(names) & df['Black'].isin(names)]

In [ ]:
def game_model(data):
    